In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
!wget https://www.dropbox.com/s/pjkn1icctxrc2en/Mask_Data.zip?dl=0

--2022-06-09 00:52:14--  https://www.dropbox.com/s/pjkn1icctxrc2en/Mask_Data.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pjkn1icctxrc2en/Mask_Data.zip [following]
--2022-06-09 00:52:14--  https://www.dropbox.com/s/raw/pjkn1icctxrc2en/Mask_Data.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6d5a689712be7355ac49a42ba6.dl.dropboxusercontent.com/cd/0/inline/Bm3TeI969OCcDcNJ6zzDLfwgDvLj1Fxy7Q5TXAXXRqEwiefeARRKcibaCoGIPNCCQjVd5GUhehFvHBqJLNyO_pqVcsoz_zE31KE2-TcdwSqXrxbXU6AHXE_ZPtDuWElaMpONoD9aus05Cs_81Hv5NdP53eyilxeNk0Cnbg6Nhu4dZQ/file# [following]
--2022-06-09 00:52:14--  https://uc6d5a689712be7355ac49a42ba6.dl.dropboxusercontent.com/cd/0/inline/Bm3TeI969OCcDcNJ6zzDLfwgDvLj1Fxy7Q5TXAXXRqEwiefeARRKcibaCo

In [3]:
!mv /content/Mask_Data.zip?dl=0 /content/Mask_Data.zip

!unzip /content/Mask_Data.zip

Archive:  /content/Mask_Data.zip
   creating: Mask_Data/cloth/
  inflating: Mask_Data/cloth/1.png   
  inflating: Mask_Data/cloth/1022.png  
  inflating: Mask_Data/cloth/1025.png  
  inflating: Mask_Data/cloth/1053.png  
  inflating: Mask_Data/cloth/1056.png  
  inflating: Mask_Data/cloth/1058.png  
  inflating: Mask_Data/cloth/1059.png  
  inflating: Mask_Data/cloth/1069.png  
  inflating: Mask_Data/cloth/1081.png  
  inflating: Mask_Data/cloth/1090.png  
  inflating: Mask_Data/cloth/111.png  
  inflating: Mask_Data/cloth/1126.png  
  inflating: Mask_Data/cloth/1144.png  
  inflating: Mask_Data/cloth/1145.png  
  inflating: Mask_Data/cloth/1152.png  
  inflating: Mask_Data/cloth/1156.png  
  inflating: Mask_Data/cloth/1158.png  
  inflating: Mask_Data/cloth/1186.png  
  inflating: Mask_Data/cloth/1193.png  
  inflating: Mask_Data/cloth/1209.png  
  inflating: Mask_Data/cloth/1224.png  
  inflating: Mask_Data/cloth/1232.png  
  inflating: Mask_Data/cloth/1234.png  
  inflating: Mask_Da

In [4]:
import os

class_dirs = [x[0] for x in os.walk('/content/Mask_Data')][1:]


root_dir = '/content/masks'

val_path = root_dir + '/mask_val'
test_path = root_dir + '/mask_test'
train_path = root_dir + '/mask_train'

if not os.path.isdir(root_dir):
    os.mkdir(root_dir)

if not os.path.isdir(val_path):
    os.mkdir(val_path)

if not os.path.isdir(test_path):
    os.mkdir(test_path)

if not os.path.isdir(train_path):
    os.mkdir(train_path)  

In [6]:
for cls in class_dirs:
    if not os.path.isdir(val_path + '/' + cls.split('/')[-1]):
        os.mkdir(val_path + '/' + cls.split('/')[-1])

    if not os.path.isdir(test_path + '/' + cls.split('/')[-1]):
        os.mkdir(test_path + '/' + cls.split('/')[-1])

    if not os.path.isdir(train_path + '/' + cls.split('/')[-1]):
        os.mkdir(train_path + '/' + cls.split('/')[-1]) 

In [7]:
from os import listdir
from os.path import isfile, join
import random
import shutil

for clss in class_dirs:
    #print(clss)
    onlyfiles = [join(clss, f) for f in listdir(clss) if isfile(join(clss, f))]
    random.shuffle(onlyfiles)
    
    #print(onlyfiles)
    files_len = len(onlyfiles)

    train_split = 0.7
    test_split = 0.15
    val_split = 0.15

    train_splt = onlyfiles[:int(train_split*files_len)]
    test_splt = onlyfiles[int(train_split*files_len): int(train_split*files_len) + int(test_split*files_len)]
    val_split = onlyfiles[int(train_split*files_len) + int(test_split*files_len) :]

    #for train_inst in train_splt:
    for img_fl in train_splt:
        final_dir = '/content/masks/mask_train'
        mask_class = img_fl.split('/')[-2]
        img_name = img_fl.split('/')[-1]

        shutil.copy(img_fl, final_dir + '/' + mask_class + '/' + img_name)

    for img_fl in test_splt:
        final_dir = '/content/masks/mask_test'
        mask_class = img_fl.split('/')[-2]
        img_name = img_fl.split('/')[-1]

        shutil.copy(img_fl, final_dir + '/' + mask_class + '/' + img_name)


    for img_fl in val_split:
        final_dir = '/content/masks/mask_val'
        mask_class = img_fl.split('/')[-2]
        img_name = img_fl.split('/')[-1]

        shutil.copy(img_fl, final_dir + '/' + mask_class + '/' + img_name)







In [8]:
import PIL

train_transform = transforms.Compose(
    [transforms.Resize((124,124)),
     transforms.ColorJitter(hue=.05, saturation=.05),
     transforms.RandomHorizontalFlip(),
     transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_transform = transforms.Compose(
    [transforms.Resize((124,124)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32




train_dataset = torchvision.datasets.ImageFolder(root = train_path, transform=train_transform)
test_dataset = torchvision.datasets.ImageFolder(root = test_path, transform=test_transform)
val_dataset = torchvision.datasets.ImageFolder(root = val_path, transform=test_transform)


#train_set, val_set, test_set = torch.utils.data.random_split(dataset, [int(0.7 * data_len), int(0.15 * data_len), int(data_len - (int(0.7 * data_len)+ int(0.15 * data_len)))])



trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

valloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('cloth', 'n95', 'surgical', 'mask_weared_incorrect', 'without_mask')


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1293: UserWarning: The parameter 'resample' is deprecated since 0.12 and will be removed 0.14. Please use 'interpolation' instead.
  "The parameter 'resample' is deprecated since 0.12 and will be removed 0.14. "


In [9]:
print("Number of data in train set = " + str(train_dataset.__len__()))
print("Number of data in test set = " + str(test_dataset.__len__()))
print("Number of data in val set = " + str(val_dataset.__len__()))

Number of data in train set = 1575
Number of data in test set = 335
Number of data in val set = 340


In [12]:
## Code inspired by https://github.com/kuangliu/pytorch-cifar/blob/master/models/resnet.py

import torch.nn as nn
import torch.nn.functional as F
import torch


class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False), nn.BatchNorm2d(self.expansion*planes))
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(nn.Conv2d(in_planes, self.expansion*planes,kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes))
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [13]:
model = ResNet18()

In [14]:
# Setup the device to run the computations
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device::', device)

Device:: cuda:0


In [15]:
#in_f = model.linear.in_features
num_class = len(classes)

model.linear = nn.Linear(in_features = 8192, out_features = num_class)

model = model.to(device)

In [16]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [17]:
num_epoch = 50

In [18]:
import tqdm

# Function to save the model
def saveModel():
    path = "best_model.pth"
    torch.save(model.state_dict(), path)

# Function to test the model with the test dataset and print the accuracy for the test images
eval_losses=[]
eval_accu=[]
 
def test(epoch):
    model.eval()
 
    running_loss=0
    correct=0
    total=0
 
    with torch.no_grad():
        for data in tqdm.tqdm(testloader):
            images,labels=data[0].to(device),data[1].to(device)
       
            outputs=model(images)
 
            loss= criterion(outputs,labels)
            running_loss+=loss.item()
       
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
   
        test_loss=running_loss/len(testloader)
        accu=100.*correct/total
 
        eval_losses.append(test_loss)
        eval_accu.append(accu)
 
        print('Test Loss: %.3f | Accuracy: %.3f'%(test_loss,accu)) 
    return(test_loss, accu)


train_accu = []
train_losses = []
test_accu = []
test_losses = []
best_accuracy = 0.0

def train(epoch, best_accuracy):
    print('\nEpoch : %d'%epoch)
   
    model.train()
 
    running_loss=0
    correct=0
    total=0
 
    for data in tqdm.tqdm(trainloader):
     
      inputs,labels=data[0].to(device),data[1].to(device)
     
      optimizer.zero_grad()
      outputs=model(inputs)
      loss=criterion(outputs,labels)
      loss.backward()
      optimizer.step()
 
      running_loss += loss.item()


      
     
      _, predicted = outputs.max(1)
      total += labels.size(0)
      correct += predicted.eq(labels).sum().item()
       
    train_loss=running_loss/len(trainloader)
    accu=100.*correct/total
    test_loss, test_accuracy = test(epoch)
   
    train_accu.append(accu)
    train_losses.append(train_loss)
    test_accu.append(test_accuracy)
    test_losses.append(test_loss)
    print('Train Loss: %.3f | Accuracy: %.3f | Val_accuracy: %.3f | Val_loss: %.3f'%(train_loss,accu,test_accuracy,test_loss))
    if accu > best_accuracy:
        saveModel()
        best_accuracy = accu

    return best_accuracy

In [ ]:
for epoch in range(num_epoch):
    best_accuracy = train(epoch, best_accuracy)


Epoch : 0


100%|██████████| 11/11 [00:01<00:00,  8.67it/s]


Test Loss: 1.442 | Accuracy: 60.000
Train Loss: 1.161 | Accuracy: 59.111 | Val_accuracy: 60.000 | Val_loss: 1.442

Epoch : 1


100%|██████████| 11/11 [00:01<00:00,  8.77it/s]


Test Loss: 0.736 | Accuracy: 76.418
Train Loss: 0.933 | Accuracy: 69.968 | Val_accuracy: 76.418 | Val_loss: 0.736

Epoch : 2


 84%|████████▍ | 42/50 [00:14<00:02,  2.90it/s]

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
loss_stats = {'train': train_losses, 'val': test_losses}
accuracy_stats = {'train': train_accu, 'val': test_accu}

train_val_acc_df = pd.DataFrame.from_dict(accuracy_stats).reset_index().melt(id_vars=['index']).rename(columns={"index":"epochs"})
train_val_loss_df = pd.DataFrame.from_dict(loss_stats).reset_index().melt(id_vars=['index']).rename(columns={"index":"epochs"})

# Plot line charts
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(30,10))
sns.lineplot(data=train_val_acc_df, x = "epochs", y="value", hue="variable",  ax=axes[0]).set_title('Train-Val Accuracy/Epoch')
sns.lineplot(data=train_val_loss_df, x = "epochs", y="value", hue="variable", ax=axes[1]).set_title('Train-Val Loss/Epoch')

In [ ]:
y_pred_list = []
y_true_list = []
model.eval()
with torch.no_grad():
    for x_batch, y_batch in tqdm(testloader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = model(x_batch)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_true_list.append(y_batch.cpu().numpy())

In [ ]:
y_pred_final = []
y_true_final = []
for i in y_pred_list:
    y_pred_final = y_pred_final + list(i)

for i in y_true_list:
    y_true_final = y_true_final + list(i)

In [ ]:
print(classification_report(y_true_final, y_pred_final))


In [ ]:
print(confusion_matrix(y_true_final, y_pred_final))

In [ ]:
torch.save(model.state_dict(), 'main_model.pth')


In [ ]:
import seaborn as sns
sns.heatmap(confusion_matrix(y_true_final, y_pred_final), annot=True,  classes)

The model will be running on cuda:0 device
For epoch 1 the test accuracy over the whole test set is 97 %
For epoch 2 the test accuracy over the whole test set is 98 %
For epoch 3 the test accuracy over the whole test set is 98 %
For epoch 4 the test accuracy over the whole test set is 98 %
For epoch 5 the test accuracy over the whole test set is 98 %
For epoch 6 the test accuracy over the whole test set is 98 %
For epoch 7 the test accuracy over the whole test set is 98 %
For epoch 8 the test accuracy over the whole test set is 98 %
For epoch 9 the test accuracy over the whole test set is 98 %
For epoch 10 the test accuracy over the whole test set is 96 %
For epoch 11 the test accuracy over the whole test set is 98 %
For epoch 12 the test accuracy over the whole test set is 98 %
For epoch 13 the test accuracy over the whole test set is 98 %
For epoch 14 the test accuracy over the whole test set is 98 %
For epoch 15 the test accuracy over the whole test set is 98 %
For epoch 16 the test